# Hugging Face

In [1]:
from flair.data import Sentence
from flair.models import SequenceTagger
from collections import defaultdict
import numpy as np

def load_conllu(filepath):
    sentences = []
    current_sentence = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            if line.startswith('#') or line.strip() == "":
                if current_sentence:
                    sentences.append(current_sentence)
                    current_sentence = []
                continue
            parts = line.split('\t')
            form, upos = parts[1], parts[3]  # FORM = index 1, UPOS = index 3
            current_sentence.append((form, upos))
    return sentences

# Evaluation 
def evaluate_flair_model(test_data, model):
    true_labels = []
    pred_labels = []
    sentence_accuracy = []
    class_metrics = defaultdict(lambda: {"tp": 0, "fp": 0, "fn": 0})

    for sentence in test_data:
        words = [token[0] for token in sentence]
        true_pos = [token[1] for token in sentence]
        flair_sentence = Sentence(" ".join(words))
        
        #Prédire les POS tags
        model.predict(flair_sentence)
        pred_pos = [label.value for label in flair_sentence.get_labels()]
        
        true_labels.extend(true_pos)
        pred_labels.extend(pred_pos)

        sentence_accuracy.append(int(true_pos == pred_pos))

        for t, p in zip(true_pos, pred_pos):
            if t == p:
                class_metrics[t]["tp"] += 1
            else:
                class_metrics[t]["fn"] += 1
                class_metrics[p]["fp"] += 1

    for label, metrics in class_metrics.items():
        tp = metrics["tp"]
        fp = metrics["fp"]
        fn = metrics["fn"]
        precision = (tp / (tp + fp) * 100) if (tp + fp) > 0 else 0
        recall = (tp / (tp + fn) * 100) if (tp + fn) > 0 else 0
        f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0
        print(f"Class {label} - Precision: {precision:.2f}%, Recall: {recall:.2f}%, F1: {f1:.2f}%")

    f1_macro = np.mean([2 * m["tp"] / (2 * m["tp"] + m["fp"] + m["fn"]) * 100 for m in class_metrics.values()])
    
    total_tp = sum(m["tp"] for m in class_metrics.values())
    total_fp = sum(m["fp"] for m in class_metrics.values())
    total_fn = sum(m["fn"] for m in class_metrics.values())
    precision_micro = total_tp / (total_tp + total_fp) * 100
    recall_micro = total_tp / (total_tp + total_fn) * 100
    f1_micro = 2 * precision_micro * recall_micro / (precision_micro + recall_micro)

    print(f"Sentence accuracy: {np.mean(sentence_accuracy) * 100:.2f}%")
    print(f"F1-macro: {f1_macro:.2f}%")
    print(f"F1-micro: {f1_micro:.2f}%")

# Flair pré-entraîné
model = SequenceTagger.load("qanastek/pos-french-camembert-flair")

test_sentences = load_conllu('fr_sequoia-ud-test.conllu')

# Évaluation
evaluate_flair_model(test_sentences, model)


pytorch_model.bin:   0%|          | 0.00/539M [00:00<?, ?B/s]

2024-10-23 22:15:33,631 SequenceTagger predicts: Dictionary with 68 tags: O, DET, NFP, ADJFP, AUX, VPPMS, ADV, PREP, PDEMMS, NMS, COSUB, PINDMS, PPOBJMS, VERB, DETFS, NFS, YPFOR, VPPFS, PUNCT, DETMS, PROPN, ADJMS, PPER3FS, ADJFS, COCO, NMP, PREL, PPER1S, ADJMP, VPPMP, DINTMS, PPER3MS, PPER3MP, PREF, ADJ, DINTFS, CHIF, XFAMIL, PRELFS, SYM, NOUN, MOTINC, PINDFS, PPOBJMP, NUM, PREFP, PDEMFS, VPPFP, PPER3FP, PPOBJFS
Class PRON - Precision: 82.35%, Recall: 3.41%, F1: 6.56%
Class PDEMMS - Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class VERB - Precision: 52.57%, Recall: 32.78%, F1: 40.38%
Class SCONJ - Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class COSUB - Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class DET - Precision: 47.01%, Recall: 25.27%, F1: 32.87%
Class NOUN - Precision: 70.59%, Recall: 1.11%, F1: 2.19%
Class NFS - Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class ADJ - Precision: 57.14%, Recall: 1.25%, F1: 2.45%
Class ADJFS - Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class PUNCT -

## Analyse des performances
Les résultats montrent des performances globalement faibles, en grande partie à cause d'un manque d'alignement des étiquettes entre les données d'entraînement et de test :

- Classes sans détection : beaucoup de classes, comme PDEMMS, SCONJ et ADJFS, n'ont pas été reconnues du tout, avec des scores de précision et de rappel à 0.
- Précision et Rappel : même pour les classes qui ont obtenu des résultats, le rappel est très bas. Par exemple, le rappel pour PRON est seulement de 3.41 %, ce qui signifie que le modèle rate beaucoup d'étiquettes correctes.

Il est clair que les étiquettes utilisées dans le modèle pré-entraîné ne correspondent pas à celles du corpus de test. Pour obtenir une évaluation fiable et comparer les résultats avec d'autres modèles, il est essentiel de procéder à un alignement des labels (cela fait partie de la structuration des données). Cela  permettra d'améliorer les performances du modèle et d'assurer une évaluation cohérente et correcte. 